# [모듈 2.1] Inference NCF on INF2 - Benchmarking

# 1. 환경 셋업

## 1.1. 기본 세팅
사용하는 패키지는 import 시점에 다시 재로딩 합니다.

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('./src')

필요한 torch_neuronx 를 로딩 합니다.

In [2]:
import torch
import torch_neuronx

# 2. 훈련된 모델 로딩

## 훈련 모델 아티펙트 확인

- 이미 훈련된 파이토치로 훈련된 모델 아티텍트의 경로를 지정합니다.

In [3]:
artifact_path = 'models/NeuMF-end.pth'
print("model artifact is assigend from : ", artifact_path)

model artifact is assigend from :  models/NeuMF-end.pth


## 모델 로딩에 필요한 설정 파일 생성

- 모델 로딩시에 필요한 파라미터 사용 (기존의 값을 사용 함)

In [4]:
import json
import os
import config
from common_utils import save_json, load_json

class Params:
    def __init__(self):
        self.factor_num = 32
        self.num_layers = 3
        self.dropout = 0.0
                        
args = Params()
print("# of num_layers: ", args.num_layers)


# 모델 훈련시에 결정된 user, item 의 숫자
user_num = 6040  
item_num = 3706
print("user_num: ", user_num, " item_num: ", item_num)

model_config_dict = {
    'user_num': str(user_num),
    'item_num': str(item_num),
    'factor_num' : str(args.factor_num),
    'num_layers' : str(args.num_layers),
    'dropout' : str(args.dropout),
    'model_type': config.model
}

model_config_file = 'model_config.json'
model_config_file_path = os.path.join('src', model_config_file)

save_json(model_config_file_path, model_config_dict)
# model_config_dict = load_json(model_config_file_path)    
# model_config_dict

# of num_layers:  3
user_num:  6040  item_num:  3706
src/model_config.json is saved


'src/model_config.json'

## 모델 로딩
- 모델 로딩 함수 model_fn() 를 통하여 모델 로딩


In [5]:
from inference import model_fn

ncf_model = model_fn(config.model_path)

######## Staring model_fn() ###############
device:  cpu


# 3. 모델 컴파일

## 샘플 입력 생성

In [6]:
import numpy as np
import torch

def create_dummy_input(batch_size):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    # print("Using {} device".format(device))

    user_np = np.zeros((1,100)).astype(np.int32)
    item_np = np.random.randint(low=1, high=1000, size=(1,100)).astype(np.int32)

    return (
        torch.repeat_interleave(torch.from_numpy(user_np), batch_size, 0),
        torch.repeat_interleave(torch.from_numpy(item_np), batch_size, 0),
    )

dummy_inputs = create_dummy_input(batch_size=1)

print("type: ", type(dummy_inputs))
print("len: ", len(dummy_inputs))


type:  <class 'tuple'>
len:  2


## Torch Script 으로 변환 (컴파일)

In [7]:
def convert_torch_script(model, dummy_inputs):
    # Compile the model for Neuron
    model_neuron = torch_neuronx.trace(model, dummy_inputs)
    
    return model_neuron

model_neuron = convert_torch_script(ncf_model, dummy_inputs)

#### prediction: 
 tensor([[[-1.1848],
         [-2.9062],
         [-2.9215],
         [-0.2536],
         [-2.1842],
         [-0.9262],
         [ 1.8365],
         [-1.9201],
         [-3.1589],
         [-1.3983],
         [-0.9339],
         [-2.1668],
         [ 0.6848],
         [-3.3775],
         [ 0.6939],
         [-2.4077],
         [ 1.1072],
         [-3.1092],
         [-3.4677],
         [-3.0932],
         [-3.1637],
         [ 0.8479],
         [ 3.2808],
         [-2.6363],
         [-1.3871],
         [-4.2089],
         [-0.5071],
         [-0.6448],
         [-2.0258],
         [-1.0498],
         [-4.3857],
         [ 0.5838],
         [-2.1706],
         [-3.4535],
         [ 1.3635],
         [-3.5763],
         [-1.6100],
         [-1.7371],
         [-1.5105],
         [ 1.6561],
         [-1.6870],
         [-3.5930],
         [-3.4934],
         [-3.4258],
         [-1.9256],
         [-2.4687],
         [-4.3822],
         [-2.5416],
         [-1.7941],
 

/home/ubuntu/aws_neuron_venv_pytorch/lib/python3.8/site-packages/torch_neuronx/xla_impl/hlo_conversion.py:117: UserWarning: Received an input tensor that was unused. Tensor will be ignored. (index=0, shape=torch.Size([1, 100]), dtype=torch.int32)
  warnings.warn(
/home/ubuntu/aws_neuron_venv_pytorch/lib/python3.8/site-packages/torch_neuronx/xla_impl/hlo_conversion.py:117: UserWarning: Received an input tensor that was unused. Tensor will be ignored. (index=1, shape=torch.Size([1, 100]), dtype=torch.int32)
  warnings.warn(
03/12/2023 06:40:17 AM WARNING 36012 [py.warnings]: /home/ubuntu/aws_neuron_venv_pytorch/bin/neuronx-cc:8: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecati

# 4. 모델 추론

In [8]:
def extract_top_k(prediction, top_k = 10):
    prediction = torch.squeeze(prediction) # remove dimension
    _, indices = torch.topk(prediction, top_k)
    
    return indices

prediction = model_neuron(dummy_inputs[0],dummy_inputs[1])
print("type:prediction ", type(prediction))
print("type:prediction[0] ", type(prediction[0]))

recommended_item_index = extract_top_k(prediction[0], top_k = 10)
print("recommended_item_index:  \n", recommended_item_index)

type:prediction  <class 'tuple'>
type:prediction[0]  <class 'torch.Tensor'>
recommended_item_index:  
 tensor([22, 87, 70,  6, 39, 34, 16, 92, 21, 78])


# 5. 모델 저장 및 로딩 후 테스트

In [9]:
# Save the TorchScript for inference deployment
filename = 'models/model.pt'
torch.jit.save(model_neuron, filename)

In [10]:
# Load the TorchScript compiled model
load_model_neuron = torch.jit.load(filename)

prediction = load_model_neuron(dummy_inputs[0],dummy_inputs[1])
print("type:prediction ", type(prediction))
print("type:prediction[0] ", type(prediction[0]))

recommended_item_index = extract_top_k(prediction[0], top_k = 10)
print("recommended_item_index:  \n", recommended_item_index)


type:prediction  <class 'tuple'>
type:prediction[0]  <class 'torch.Tensor'>
recommended_item_index:  
 tensor([22, 87, 70,  6, 39, 34, 16, 92, 21, 78])


# 6. 벤치 마킹


## 일부 샘플로 추론 시간 측정

In [11]:
import time
latencies = []
num_test = 5
for _ in range(num_test):
    start = time.time()
    prediction = load_model_neuron(dummy_inputs[0],dummy_inputs[1])    
    finish = time.time()
    elapse_time = round((finish - start) * 1000, 5)
    # print(prediction)
    latencies.append(elapse_time)

print("latencies: ", latencies)

latencies:  [0.48184, 0.12088, 0.11539, 0.11706, 0.11206]


## 벤치 마킹, 모델 수 및 Thread 수 조절

In [12]:
from inf2_util import benchmark

In [13]:
# Benchmark BERT on Neuron
# benchmark(filename, example, n_models=2, n_threads=2, batches_per_thread=1000)
benchmark(filename, dummy_inputs)

Filename:    models/model.pt
Batch Size:  1
Batches:     2000
Inferences:  2000
Threads:     2
Models:      2
Duration:    0.115
Throughput:  17406.568
Latency P50: 0.111
Latency P95: 0.125
Latency P99: 0.135


In [14]:
benchmark(filename, dummy_inputs, n_models=2, n_threads=4, batches_per_thread=1000)

Filename:    models/model.pt
Batch Size:  1
Batches:     4000
Inferences:  4000
Threads:     4
Models:      2
Duration:    0.142
Throughput:  28144.308
Latency P50: 0.136
Latency P95: 0.161
Latency P99: 0.206


## 최적의 배치 사이즈 찾기

In [15]:
# Compile BERT for different batch sizes
for batch_size in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
# for batch_size in [1, 2, 4, 8, 16, 32, 64, 128, 256]:
#     tokenizer = AutoTokenizer.from_pretrained(name)
#     model = AutoModelForSequenceClassification.from_pretrained(name, torchscript=True)
    dummy_inputs = create_dummy_input(batch_size= batch_size)
    # example = encode(tokenizer, sequence_0, sequence_2, batch_size=batch_size)
    model_neuron = convert_torch_script(ncf_model, dummy_inputs)    
    # ㅠmodel_neuron = torch_neuronx.trace(model, example)
    filename = f'model_batch_size_{batch_size}.pt'
    torch.jit.save(model_neuron, filename)

#### prediction: 
 tensor([[[-1.9389],
         [-1.9872],
         [-1.2366],
         [ 0.6196],
         [-2.4418],
         [-3.1095],
         [-2.8314],
         [-1.5271],
         [-2.1842],
         [-1.4032],
         [ 0.2330],
         [-0.2536],
         [ 0.6151],
         [-0.7603],
         [-1.0449],
         [-0.7522],
         [-0.8561],
         [-4.0481],
         [-2.9439],
         [-2.3725],
         [ 0.1770],
         [-2.3422],
         [-2.0962],
         [-2.0046],
         [ 1.3567],
         [-3.0413],
         [ 1.1072],
         [-0.4168],
         [-3.5763],
         [ 0.9679],
         [ 1.8365],
         [-1.1425],
         [-2.1949],
         [-3.2080],
         [-2.5564],
         [-0.1525],
         [-3.4258],
         [-2.5250],
         [-0.1535],
         [-3.1450],
         [-3.6925],
         [-2.5250],
         [-1.1053],
         [-1.5729],
         [-1.6448],
         [-1.3120],
         [-1.4801],
         [-1.6100],
         [-3.0187],
 

03/12/2023 06:40:23 AM WARNING 36081 [py.warnings]: /home/ubuntu/aws_neuron_venv_pytorch/bin/neuronx-cc:8: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  sys.exit(main())

03/12/2023 06:40:25 AM WARNING 36081 [WalrusDriver]: 0% PSUM demand before spilling
03/12/2023 06:40:25 AM WARNING 36081 [WalrusDriver]: spilling from PSUM cost about 0 cycles
03/12/2023 06:40:25 AM WARNING 36081 [WalrusDriver]: 0% PSUM utilization after allocation
03/12/2023 06:40:25 AM WARNING 36081 [WalrusDriver]: spilling from SB cost about 0 cycles
03/12/2023 06:40:25 AM WARNING 36081 [WalrusDriver]: 0 bytes/partition (0%) successfully pinned
03/12/2023 06:40:25 AM WARNING 36081 [WalrusDriver]:

#### prediction: 
 tensor([[[-1.0365],
         [-0.6960],
         [-5.3186],
         [-2.5546],
         [-1.9959],
         [-1.6822],
         [-2.6483],
         [-2.0274],
         [-0.5445],
         [-3.1637],
         [-1.0943],
         [-3.0403],
         [-1.6977],
         [-0.2422],
         [-1.7862],
         [-0.4970],
         [-2.5605],
         [-1.3185],
         [-4.0969],
         [-2.1618],
         [-2.4952],
         [-3.4677],
         [-2.5342],
         [-3.0927],
         [ 0.4353],
         [-2.5310],
         [-3.4284],
         [ 0.3642],
         [-2.0267],
         [-0.7425],
         [-3.5763],
         [-0.5675],
         [-0.9022],
         [-1.5686],
         [-0.6397],
         [-2.1833],
         [ 2.1892],
         [-2.0686],
         [-1.0002],
         [-4.4377],
         [ 2.2862],
         [-1.8735],
         [-0.9339],
         [-2.0422],
         [-1.0282],
         [-0.6058],
         [-2.7894],
         [ 0.6939],
         [-3.1095],
 

/home/ubuntu/aws_neuron_venv_pytorch/lib/python3.8/site-packages/torch_neuronx/xla_impl/hlo_conversion.py:117: UserWarning: Received an input tensor that was unused. Tensor will be ignored. (index=0, shape=torch.Size([2, 100]), dtype=torch.int32)
  warnings.warn(
/home/ubuntu/aws_neuron_venv_pytorch/lib/python3.8/site-packages/torch_neuronx/xla_impl/hlo_conversion.py:117: UserWarning: Received an input tensor that was unused. Tensor will be ignored. (index=1, shape=torch.Size([2, 100]), dtype=torch.int32)
  warnings.warn(
03/12/2023 06:40:26 AM WARNING 36126 [py.warnings]: /home/ubuntu/aws_neuron_venv_pytorch/bin/neuronx-cc:8: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecati

#### prediction: 
 tensor([[[-2.5242],
         [-0.4542],
         [-2.5238],
         [ 0.1247],
         [-1.4890],
         [-1.0365],
         [-2.7635],
         [-3.4934],
         [ 0.2207],
         [-2.1106],
         [-2.1096],
         [-3.7962],
         [-3.0040],
         [-0.7806],
         [-0.9262],
         [-3.7332],
         [-3.4024],
         [-0.2246],
         [-3.0406],
         [-0.2422],
         [ 2.2073],
         [-0.5709],
         [-3.2271],
         [-0.0759],
         [-0.0950],
         [ 0.5289],
         [-5.0949],
         [ 1.1824],
         [-2.8489],
         [-0.1097],
         [ 1.0631],
         [-1.4877],
         [-1.5194],
         [-1.3754],
         [-1.0986],
         [-2.6928],
         [ 0.8927],
         [-2.2033],
         [-0.4616],
         [-1.3871],
         [-2.2334],
         [-3.9233],
         [-1.7654],
         [ 0.3776],
         [-2.0860],
         [-2.4765],
         [-0.9894],
         [-2.6483],
         [-2.5493],
 

...

/home/ubuntu/aws_neuron_venv_pytorch/lib/python3.8/site-packages/torch_neuronx/xla_impl/hlo_conversion.py:117: UserWarning: Received an input tensor that was unused. Tensor will be ignored. (index=0, shape=torch.Size([8, 100]), dtype=torch.int32)
  warnings.warn(
/home/ubuntu/aws_neuron_venv_pytorch/lib/python3.8/site-packages/torch_neuronx/xla_impl/hlo_conversion.py:117: UserWarning: Received an input tensor that was unused. Tensor will be ignored. (index=1, shape=torch.Size([8, 100]), dtype=torch.int32)
  warnings.warn(
03/12/2023 06:40:45 AM WARNING 36398 [py.warnings]: /home/ubuntu/aws_neuron_venv_pytorch/bin/neuronx-cc:8: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecati

#### prediction: 
 tensor([[[-2.1843],
         [ 0.2960],
         [-1.2366],
         [-3.8482],
         [-0.9900],
         [-0.5445],
         [-3.1589],
         [ 1.0348],
         [-3.5930],
         [-0.2443],
         [-1.4032],
         [-0.4616],
         [ 0.5075],
         [-2.7533],
         [-2.0439],
         [-2.0175],
         [-2.4687],
         [-1.4890],
         [-1.7728],
         [-2.7092],
         [-1.1757],
         [-3.5692],
         [-2.4353],
         [-2.4068],
         [-0.3075],
         [-0.3596],
         [-1.7533],
         [ 0.6629],
         [-4.0188],
         [ 2.1306],
         [ 0.2117],
         [-2.1862],
         [-2.3422],
         [-2.6509],
         [-1.1435],
         [-0.8315],
         [-2.6332],
         [-1.7654],
         [ 0.9255],
         [ 0.3642],
         [-1.1435],
         [-2.5310],
         [-1.2659],
         [-1.6448],
         [-2.7542],
         [-4.7241],
         [-1.2374],
         [ 2.4899],
         [-1.6553],
 

/home/ubuntu/aws_neuron_venv_pytorch/lib/python3.8/site-packages/torch_neuronx/xla_impl/hlo_conversion.py:117: UserWarning: Received an input tensor that was unused. Tensor will be ignored. (index=0, shape=torch.Size([9, 100]), dtype=torch.int32)
  warnings.warn(
/home/ubuntu/aws_neuron_venv_pytorch/lib/python3.8/site-packages/torch_neuronx/xla_impl/hlo_conversion.py:117: UserWarning: Received an input tensor that was unused. Tensor will be ignored. (index=1, shape=torch.Size([9, 100]), dtype=torch.int32)
  warnings.warn(
03/12/2023 06:40:48 AM WARNING 36444 [py.warnings]: /home/ubuntu/aws_neuron_venv_pytorch/bin/neuronx-cc:8: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecati

#### prediction: 
 tensor([[[ 1.4956],
         [-3.1032],
         [-3.7962],
         [-1.7941],
         [-2.6515],
         [-0.7275],
         [-0.5078],
         [-2.7542],
         [ 0.6848],
         [-0.5196],
         [-1.2194],
         [-0.6477],
         [-4.0309],
         [-3.2231],
         [ 0.3195],
         [-2.9161],
         [-1.2807],
         [-3.0826],
         [-1.7549],
         [-3.4934],
         [-4.5206],
         [-2.7994],
         [-2.1908],
         [ 0.4353],
         [-0.5231],
         [-1.3301],
         [-2.0933],
         [-0.2389],
         [-1.8741],
         [ 2.2073],
         [-1.1898],
         [-4.5842],
         [-0.0635],
         [-2.5310],
         [-0.5940],
         [-2.5539],
         [-3.2271],
         [-1.5556],
         [-1.8003],
         [ 1.3003],
         [-1.3663],
         [-1.8576],
         [ 2.2073],
         [ 1.0348],
         [-1.0946],
         [-2.7866],
         [ 1.3635],
         [-2.7542],
         [-3.3392],
 

/home/ubuntu/aws_neuron_venv_pytorch/lib/python3.8/site-packages/torch_neuronx/xla_impl/hlo_conversion.py:117: UserWarning: Received an input tensor that was unused. Tensor will be ignored. (index=0, shape=torch.Size([10, 100]), dtype=torch.int32)
  warnings.warn(
/home/ubuntu/aws_neuron_venv_pytorch/lib/python3.8/site-packages/torch_neuronx/xla_impl/hlo_conversion.py:117: UserWarning: Received an input tensor that was unused. Tensor will be ignored. (index=1, shape=torch.Size([10, 100]), dtype=torch.int32)
  warnings.warn(
03/12/2023 06:40:52 AM WARNING 36492 [py.warnings]: /home/ubuntu/aws_neuron_venv_pytorch/bin/neuronx-cc:8: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#depreca

In [16]:
# Benchmark BERT for different batch sizes
for batch_size in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
# for batch_size in [1, 2, 4, 8, 16, 32, 64, 128, 256]:
    print('-'*50)
    dummy_inputs = create_dummy_input(batch_size= batch_size)
    filename = f'model_batch_size_{batch_size}.pt'
    benchmark(filename, dummy_inputs)
    print()

--------------------------------------------------
Filename:    model_batch_size_1.pt
Batch Size:  1
Batches:     2000
Inferences:  2000
Threads:     2
Models:      2
Duration:    0.731
Throughput:  2735.084
Latency P50: 0.738
Latency P95: 0.891
Latency P99: 1.335

--------------------------------------------------
Filename:    model_batch_size_2.pt
Batch Size:  2
Batches:     2000
Inferences:  4000
Threads:     2
Models:      2
Duration:    0.986
Throughput:  4056.616
Latency P50: 0.936
Latency P95: 1.147
Latency P99: 1.579

--------------------------------------------------
Filename:    model_batch_size_3.pt
Batch Size:  3
Batches:     2000
Inferences:  6000
Threads:     2
Models:      2
Duration:    1.119
Throughput:  5361.024
Latency P50: 1.119
Latency P95: 1.241
Latency P99: 1.965

--------------------------------------------------
Filename:    model_batch_size_4.pt
Batch Size:  4
Batches:     2000
Inferences:  8000
Threads:     2
Models:      2
Duration:    0.947
Throughput:  845